<h2>Import Essentials Package</h2>

In [15]:
import pandas as pd
from datetime import datetime
import numpy as np
import re

In [16]:


def find_duplicates(dataframe, columns_to_check):
    """
    Finds and returns duplicated values in specified columns along with their counts.
    
    Parameters:
    dataframe (pd.DataFrame): The DataFrame to check for duplicates.
    columns_to_check (list): List of column names to check for duplicates.
    
    Returns:
    pd.DataFrame: A DataFrame with duplicated values and their counts.
    """
    # Filter the DataFrame to include only rows that are duplicated based on the specified columns
    duplicates = dataframe[dataframe.duplicated(subset=columns_to_check, keep=False)]
    
    # Group by the specified columns and count occurrences
    duplicate_values = duplicates.groupby(columns_to_check).size().reset_index(name='count')
    
    return duplicate_values



def adjust_string_case(df):
    for col in df.columns:
        if df[col].dtype == 'object':  # Check if the column is of type object (which usually means strings)
            if 'club' not in col.lower():  # Exclude columns with 'club' in their name
                df[col] = df[col].str.title()
    return df



# Implementing the business logic for is_business_logic_valid
def validate_business_logic(row):
    # Check for Valid Referral Rewards - Condition 1
    if (row['referral_status'] == 'Berhasil' and 
        pd.notna(row['reward_value']) and row['reward_value'] > 0 and 
        pd.notna(row['transaction_id']) and 
        row['transaction_status'] == 'PAID' and 
        row['transaction_type'] == 'NEW' and 
        pd.notna(row['transaction_at']) and row['transaction_at'] > row['referral_at'] and 
        row['transaction_at'].month == row['referral_at'].month and 
        not row['is_deleted'] and 
        not row.get('referrer_membership_expired', False) and 
        row.get('reward_granted', False)):
        return True
    
    # Check for Valid Referral Rewards - Condition 2
    if (row['referral_status'] in ['Menunggu', 'Tidak Berhasil'] and 
        (pd.isna(row['reward_value']) or row['reward_value'] == 0)):
        return True

    # Check for Invalid Referral Rewards - Condition 1
    if (pd.notna(row['reward_value']) and row['reward_value'] > 0 and 
        row['referral_status'] != 'Berhasil'):
        return False
    
    # Check for Invalid Referral Rewards - Condition 2
    if (pd.notna(row['reward_value']) and row['reward_value'] > 0 and 
        pd.isna(row['transaction_id'])):
        return False
    
    # Check for Invalid Referral Rewards - Condition 3
    if ((pd.isna(row['reward_value']) or row['reward_value'] == 0) and 
        pd.notna(row['transaction_id']) and 
        row['transaction_status'] == 'PAID' and 
        pd.notna(row['transaction_at']) and row['transaction_at'] > row['referral_at']):
        return False
    
    # Check for Invalid Referral Rewards - Condition 4
    if (row['referral_status'] == 'Berhasil' and 
        (pd.isna(row['reward_value']) or row['reward_value'] == 0)):
        return False
    
    # Check for Invalid Referral Rewards - Condition 5
    if (pd.notna(row['transaction_at']) and 
        row['transaction_at'] < row['referral_at']):
        return False
    
    # If none of the conditions are met, return False by default
    return False



Data Loading

In [17]:

# fact table
user_referrals_df = pd.read_csv('data/user_referrals.csv')


# dimension table
user_referrals_status_df = pd.read_csv('data/user_referral_statuses.csv')
user_referral_logs_df = pd.read_csv('data/user_referral_logs.csv')
referral_rewards_df = pd.read_csv('data/referral_rewards.csv')
paid_transactions_df = pd.read_csv('data/paid_transactions.csv')
lead_log_df = pd.read_csv('data/lead_log.csv')
user_logs_df = pd.read_csv('data/user_logs.csv')



<h2> remove duplicated

lead log

In [18]:
# fix duplicated on lead logs 
# Sort by created_at and current_status to prioritize the latest status

print(find_duplicates(lead_log_df,['lead_id']))



df_sorted = lead_log_df.sort_values(by=['created_at','id'])



# Drop duplicates keeping the last occurrence
lead_log_df = lead_log_df.drop_duplicates(subset=['lead_id', 'created_at'], keep='last')

                            lead_id  count
0  08b8ab0371b6a0cb79319628505c446c      4


user log

In [19]:
print(find_duplicates(user_logs_df,['user_id']))
df_sorted = user_logs_df.sort_values(by=['user_id','membership_expired_date'])



# Drop duplicates keeping the last occurrence
user_logs_df = user_logs_df.drop_duplicates(subset=['user_id'], keep='first')

                            user_id  count
0  2c71c5d66c7e12a0b3c200ba6ed3b78e     13
1  4acfa96cb521b6963b50bcef95bee485      7
2  cadba628952bde1670ac4c0792afc28a      2
3  f2de40c8394f4ea9a6aa49965380dd8f      2


user referral log 

In [20]:
print(find_duplicates(user_referral_logs_df,['user_referral_id']))
df_sorted = user_referral_logs_df.sort_values(by=['user_referral_id', 'created_at']).groupby('user_referral_id').first().reset_index()
# Drop duplicates keeping the last occurrence
user_referral_logs_df = user_referral_logs_df.drop_duplicates(subset=['user_referral_id'], keep='first')



                   user_referral_id  count
0  7a03e01e0c17b2322845242a796adb5e     19


<h2>Ganti semua timestamp seluruh table ke bentuk timezone wib "YYYY-MM-DD HH::MM::SS"</h2>

In [21]:
user_referral_logs_df['created_at'] = pd.to_datetime(user_referral_logs_df['created_at']).dt.tz_convert('Asia/Jakarta')
user_referral_logs_df['created_at'] = user_referral_logs_df['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

user_logs_df['membership_expired_date'] = pd.to_datetime(user_logs_df['membership_expired_date'])
user_logs_df['membership_expired_date'] = user_logs_df['membership_expired_date'].dt.tz_localize('UTC')
user_logs_df['membership_expired_date'] = user_logs_df['membership_expired_date'].dt.tz_convert('Asia/Jakarta')
user_logs_df['membership_expired_date'] = user_logs_df['membership_expired_date'].dt.strftime('%Y-%m-%d %H:%M:%S')


lead_log_df['created_at'] = pd.to_datetime(lead_log_df['created_at']).dt.tz_convert('Asia/Jakarta')
lead_log_df['created_at'] = lead_log_df['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')


user_referrals_df['updated_at'] = pd.to_datetime(user_referrals_df['updated_at']).dt.tz_convert('Asia/Jakarta')
user_referrals_df['updated_at'] = user_referrals_df['updated_at'].dt.strftime('%Y-%m-%d %H:%M:%S')


paid_transactions_df['transaction_at'] = pd.to_datetime(paid_transactions_df['transaction_at']).dt.tz_convert('Asia/Jakarta')
paid_transactions_df['transaction_at'] = paid_transactions_df['transaction_at'].dt.strftime('%Y-%m-%d %H:%M:%S')


referral_rewards_df['created_at'] = pd.to_datetime(referral_rewards_df['created_at']).dt.tz_convert('Asia/Jakarta')
referral_rewards_df['created_at'] = referral_rewards_df['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')


user_referrals_status_df['created_at'] = pd.to_datetime(user_referrals_status_df['created_at']).dt.tz_convert('Asia/Jakarta')
user_referrals_status_df['created_at'] = user_referrals_status_df['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')




handle null fits then join and then String Adjustment: Initcap should apply in string value, unless the club name.
○ Source Category: Determine referral_source_category

In [22]:
fact_table = user_referrals_df.copy()

# Step 1: Drop rows where referrer_id is NaN
#fact_table = fact_table.dropna(subset=['referrer_id','transaction_id'])



# Step 3: Replace NaN in referral_reward_id with 0
#fact_table['referral_reward_id'] = fact_table['referral_reward_id'].fillna(0)




# fact_table = fact_table.merge(user_referral_logs_df, left_on='referral_id', right_on='user_referral_id', how='left')
# fact_table = fact_table.merge(lead_log_df, left_on='referee_id', right_on='lead_id', how='left')
# fact_table = fact_table.merge(referral_rewards_df, left_on='referral_reward_id', right_on='id', how='left')
# fact_table = fact_table.merge(user_logs_df, left_on='referrer_id', right_on='user_id', how='left')
# fact_table = fact_table.merge(paid_transactions_df, left_on='transaction_id', right_on='transaction_id', how='left')
# fact_table = fact_table.merge(user_referrals_status_df, left_on='user_referral_status_id', right_on='id', how='left')

fact_table = fact_table.merge(user_referral_logs_df, left_on='referral_id', right_on='user_referral_id', how='left')
fact_table = fact_table.merge(lead_log_df, left_on='referee_id', right_on='lead_id', how='left')
fact_table = fact_table.merge(referral_rewards_df, left_on='referral_reward_id', right_on='id', how='left')
fact_table = fact_table.merge(user_logs_df, left_on='referrer_id', right_on='user_id', how='left', suffixes=('_referrer', '_userlog'))
fact_table = fact_table.merge(paid_transactions_df, left_on='transaction_id', right_on='transaction_id', how='left')
fact_table = fact_table.merge(user_referrals_status_df, left_on='user_referral_status_id', right_on='id', how='left', suffixes=('_refstatus', '_status'))








fact_table= fact_table.rename(columns={'description':'referral_status'})


# Determine referral_source_category
fact_table['referral_source_category'] = fact_table.apply(
    lambda row: 'Online' if row['referral_source'] == 'User Sign Up' else (
        'Offline' if row['referral_source'] == 'Draft Transaction' else row['source_category']
    ), axis=1
)


fact_table['reward_value'] = fact_table['reward_value'].apply(lambda x: np.nan if pd.isna(x) else int(''.join(re.findall(r'\d+', str(x)))) if re.findall(r'\d+', str(x)) else np.nan)


#merged_data = adjust_string_case(fact_table)

/Users/hypefast/opt/anaconda3/envs/old/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [24]:
fact_table

,referral_at,referral_id,referee_id,referee_name,referee_phone,referral_reward_id,referral_source,referrer_id,transaction_id,updated_at,...,is_deleted,transaction_status,transaction_at,transaction_location,timezone_transaction,transaction_type,id,referral_status,created_at_status,referral_source_category
0,2024-05-01T05:17:31Z,9331c8f144dad5a3b8e4a10467b4343a,f1327c9d6d4efee6ad69e7e467b605b9,NaN,5ba638fed7578e677c7600f1038f1b77,NaN,Draft Transaction,2c71c5d66c7e12a0b3c200ba6ed3b78e,bc3a22d1b0c651d0c807a9bdaed08e8d,2024-05-01 12:17:31,...,False,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08 15:49:37,Offline
1,2024-04-22T15:04:57Z,6371079a92bcbf0c16ae5fcdf4fc9c10,12dd343d282fb7915f55982937c30b87,8ef43a9189c084778dadf266d6ee6071,ce52ad7070f305b43a784b8503dbde13,NaN,User Sign Up,2c71c5d66c7e12a0b3c200ba6ed3b78e,4c8cae052f19cea66544affc759b76ee,2024-04-29 11:04:15,...,False,NaN,NaN,NaN,NaN,NaN,2,Berhasil,2024-03-08 17:07:10,Online
2,2024-05-02T14:10:16Z,a49105b02e690472452527663559d97a,f6507a982bde1dcda0ce0867ceac66f6,fa8e148b928aac56782d5c50042aee7b,74d8b76d490b094ba0cc7f144ed16cf5,NaN,Draft Transaction,NaN,91263eaf2af17ac140c6fc23a68882d3,2024-05-02 21:10:29,...,NaN,PAID,2024-05-02 21:10:16,GREENVILLE,Asia/Jakarta,NEW,3,Tidak Berhasil,2024-03-08 17:07:10,Offline
3,2024-04-18T03:56:22Z,fcb804c8ff24e5b2974a7e965ebea5e8,919b89759e569c5f1bdfc45ba1872d6f,4f5649d2d7b3ee9406436f026094c1fe,ce52ad7070f305b43a784b8503dbde13,NaN,User Sign Up,2c71c5d66c7e12a0b3c200ba6ed3b78e,NaN,2024-05-01 09:58:58,...,False,NaN,NaN,NaN,NaN,NaN,2,Berhasil,2024-03-08 17:07:10,Online
4,2024-05-14T06:17:03Z,9e9324e6fde29bb0d230654b38ccfdd4,d1b90f8e27b5ec0d37f180aed67d76b4,2e68f7f5c8854bd2cf2b5ff55bc7e780,321b0102e766ef73b63d1bd797203c02,2.0,Draft Transaction,f2de40c8394f4ea9a6aa49965380dd8f,e05121ea99fed4f4c5224a4667bb3dad,2024-05-14 13:17:34,...,False,PAID,2024-05-14 13:17:03,ADITYAWARMAN,Asia/Jakarta,NEW,2,Berhasil,2024-03-08 17:07:10,Offline
5,2024-05-22T09:03:10Z,b6732d2ca0bc7acbc1b39cb8a04a96cd,8cba435bca358b92462a30bc999598bf,2c44e14b1d36c87097edaf78e3110392,41440157b33b9eff6c7054134a613fc0,NaN,Lead,4acfa96cb521b6963b50bcef95bee485,NaN,2024-05-22 16:03:10,...,False,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08 15:49:37,NaN
6,2024-05-13T10:23:46Z,835e990334fe52dec832043f19ddfd07,a2f1c1a62e161b8fe76498b3936d8afa,b2b3dd66bfbbfb5154c3d6287784d0c8,3c46036fcb646a14fdd6d352d2a2eafe,1.0,Draft Transaction,f2de40c8394f4ea9a6aa49965380dd8f,f192fca27d438c053db425fade152d66,2024-05-14 13:08:29,...,False,PAID,2024-05-13 17:23:46,ADITYAWARMAN,Asia/Jakarta,NEW,2,Berhasil,2024-03-08 17:07:10,Offline
7,2024-05-02T14:06:39Z,6667436490ad3c794a8f5127772d4810,38ebaae9d064d2aa29f466bdbcf9a5b2,1f927e963a35a52a8462c80214672ad0,89900ba36a8f1ff3b02163eb3ef88861,NaN,Draft Transaction,NaN,07f21743663aacb7d212c286d47a14de,2024-05-02 21:06:56,...,NaN,PAID,2024-05-02 21:06:39,BENHIL,Asia/Jakarta,NEW,3,Tidak Berhasil,2024-03-08 17:07:10,Offline
8,2024-03-20T07:48:49Z,d452feec722a5cabc03645bb089e2735,08b8ab0371b6a0cb79319628505c446c,360701f01f783634925ce89e59decc37,ce52ad7070f305b43a784b8503dbde13,1.0,Lead,cbac8ed440d53d1c6c187874c4193db2,7709d17eaec03665f58904b72c8ab444,2024-05-01 09:58:57,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08 15:49:37,Online
9,2024-05-21T09:47:11Z,a95d6a6cefee850333bb2df257cb2f79,d0f7a95f117a5b084ea18a2fcdeb543e,aeed83f1695c48689484dccb42ad29b9,7e5a9e249a06af8dc5440331aa37b878,1.0,Draft Transaction,ac7437a85f72bcef29088bf148e05eb7,0ad3f929bef2f8177db401bba99a6692,2024-05-21 16:49:45,...,False,PAID,2024-05-21 16:47:11,ADITYAWARMAN,Asia/Jakarta,NEW,2,Berhasil,2024-03-08 17:07:10,Offline


In [11]:
# fact_table['referral_at'] = pd.to_datetime(fact_table['referral_at'])
# fact_table['transaction_at'] = pd.to_datetime(fact_table['transaction_at'])

fact_table['is_business_logic_valid'] = fact_table.apply(validate_business_logic, axis=1)

In [27]:
fact_table.to_csv('sample_data.csv',index=False)

In [14]:
reward_granted_at

Index(['referral_at', 'referral_id', 'referee_id', 'referee_name',
       'referee_phone', 'referral_reward_id', 'referral_source', 'referrer_id',
       'transaction_id', 'updated_at', 'user_referral_status_id', 'id_x',
       'user_referral_id', 'source_transaction_id', 'created_at_x',
       'is_reward_granted', 'id_y', 'lead_id', 'source_category',
       'created_at_y', 'preferred_location', 'timezone_location',
       'current_status', 'id_referrer', 'reward_value', 'created_at_refstatus',
       'reward_type', 'id_userlog', 'user_id', 'name', 'phone_number',
       'homeclub', 'timezone_homeclub', 'membership_expired_date',
       'is_deleted', 'transaction_status', 'transaction_at',
       'transaction_location', 'timezone_transaction', 'transaction_type',
       'id', 'referral_status', 'created_at_status',
       'referral_source_category', 'is_business_logic_valid'],
      dtype='object')

In [ ]:
Index(['referral_at', 'referral_id', 'referee_id', 'referee_name',
       'referee_phone', 'referral_reward_id', 'referral_source', 'referrer_id',
       'transaction_id', 'updated_at', 'user_referral_status_id', 'id',
       'user_referral_id', 'source_transaction_id', 'created_at',
       'is_reward_granted', 'id', 'lead_id', 'source_category', 'created_at',
       'preferred_location', 'timezone_location', 'current_status', 'id',
       'reward_value', 'created_at', 'reward_type', 'id', 'user_id', 'name',
       'phone_number', 'homeclub', 'timezone_homeclub',
       'membership_expired_date', 'is_deleted', 'transaction_id',
       'transaction_status', 'transaction_at', 'transaction_location',
       'timezone_transaction', 'transaction_type', 'id', 'referral_status',
       'created_at', 'referral_source_category'],
      dtype='object')




ndex(['referral_at', 'referral_id', 'referee_id', 'referee_name',
       'referee_phone', 'referral_reward_id', 'referral_source', 'referrer_id',
       'transaction_id', 'updated_at', 'user_referral_status_id', 'id_x',
       'user_referral_id', 'source_transaction_id', 'created_at_x',
       'is_reward_granted', 'id_y', 'lead_id', 'source_category',
       'created_at_y', 'preferred_location', 'timezone_location',
       'current_status', 'id_referrer', 'reward_value', 'created_at_refstatus',
       'reward_type', 'id_userlog', 'user_id', 'name', 'phone_number',
       'homeclub', 'timezone_homeclub', 'membership_expired_date',
       'is_deleted', 'transaction_status', 'transaction_at',
       'transaction_location', 'timezone_transaction', 'transaction_type',
       'id', 'referral_status', 'created_at_status',
       'referral_source_category', 'is_business_logic_valid'],
      dtype='object')